## US National Parks & Yelp! Reviews

In [20]:
# dependencies
import pandas as pd
from sqlalchemy import create_engine
import requests
import json
from pprint import pprint
# API and DB Credentials
# from keys import db_cred
from keys import gkey



### Extract Park Data from JSON


In [4]:
json_file = "resources/data.json"
parks_df = pd.read_json(json_file)
parks_df.head()

,area,coordinates,date_established_readable,date_established_unix,description,image,nps_link,states,title,id,visitors,world_heritage_site
0,"{'acres': '49,057.36', 'square_km': '198.5'}","{'latitude': 44.35, 'longitude': -68.21}","February 26, 1919",-1604599200,Covering most of Mount Desert Island and other...,"{'url': 'acadia.jpg', 'attribution': 'PixelBay...",https://www.nps.gov/acad/index.htm,"[{'id': 'state_maine', 'title': 'Maine'}]",Acadia,park_acadia,"3,303,393",False
1,"{'acres': '8,256.67', 'square_km': '33.4'}","{'latitude': -14.25, 'longitude': -170.68}","October 31, 1988",594280800,The southernmost National Park is on three Sam...,"{'url': 'american-samoa.jpg', 'attribution': '...",https://www.nps.gov/npsa/index.htm,"[{'id': 'state_american-samoa', 'title': 'Amer...",American Samoa,park_american-samoa,"28,892",False
2,"{'acres': '76,678.98', 'square_km': '310.3'}","{'latitude': 38.68, 'longitude': -109.57}","November 12, 1971",58773600,"This site features more than 2,000 natural san...","{'url': 'arches.jpg', 'attribution': 'PixelBay...",https://www.nps.gov/arch/index.htm,"[{'id': 'state_utah', 'title': 'Utah'}]",Arches,park_arches,"1,585,718",False
3,"{'acres': '242,755.94', 'square_km': '982.4'}","{'latitude': 43.75, 'longitude': -102.5}","November 10, 1978",279525600,"The Badlands are a collection of buttes, pinna...","{'url': 'badlands.jpg', 'attribution': 'PixelB...",https://www.nps.gov/badl/index.htm,"[{'id': 'state_south-dakota', 'title': 'South ...",Badlands,park_badlands,"996,263",False
4,"{'acres': '801,163.21', 'square_km': '3,242.2'}","{'latitude': 29.25, 'longitude': -103.25}","June 12, 1944",-806439600,Named for the prominent bend in the Rio Grande...,"{'url': 'big-bend.jpg', 'attribution': 'PixelB...",https://www.nps.gov/bibe/index.htm,"[{'id': 'state_texas', 'title': 'Texas'}]",Big Bend,park_big-bend,"388,290",False


### Transform Park Data

In [10]:
natl_parks=parks_df.drop(columns=['date_established_unix','image', 'nps_link','states'])
natl_parks.set_index('title',drop=True,)


,area,coordinates,date_established_readable,description,id,visitors,world_heritage_site
title,,,,,,,
Acadia,"{'acres': '49,057.36', 'square_km': '198.5'}","{'latitude': 44.35, 'longitude': -68.21}","February 26, 1919",Covering most of Mount Desert Island and other...,park_acadia,"3,303,393",False
American Samoa,"{'acres': '8,256.67', 'square_km': '33.4'}","{'latitude': -14.25, 'longitude': -170.68}","October 31, 1988",The southernmost National Park is on three Sam...,park_american-samoa,"28,892",False
Arches,"{'acres': '76,678.98', 'square_km': '310.3'}","{'latitude': 38.68, 'longitude': -109.57}","November 12, 1971","This site features more than 2,000 natural san...",park_arches,"1,585,718",False
Badlands,"{'acres': '242,755.94', 'square_km': '982.4'}","{'latitude': 43.75, 'longitude': -102.5}","November 10, 1978","The Badlands are a collection of buttes, pinna...",park_badlands,"996,263",False
Big Bend,"{'acres': '801,163.21', 'square_km': '3,242.2'}","{'latitude': 29.25, 'longitude': -103.25}","June 12, 1944",Named for the prominent bend in the Rio Grande...,park_big-bend,"388,290",False
Biscayne,"{'acres': '172,971.11', 'square_km': '700.0'}","{'latitude': 25.65, 'longitude': -80.08}","June 28, 1980","Located in Biscayne Bay, this park at the nort...",park_biscayne,"514,709",False
Black Canyon of the Gunnison,"{'acres': '30,749.75', 'square_km': '124.4'}","{'latitude': 38.57, 'longitude': -107.72}","October 21, 1999",The park protects a quarter of the Gunnison Ri...,park_black-canyon-of-the-gunnison,"238,018",False
Bryce Canyon,"{'acres': '35,835.08', 'square_km': '145.0'}","{'latitude': 37.57, 'longitude': -112.18}","February 25, 1928",Bryce Canyon is a geological amphitheater on t...,park_bryce-canyon,"2,365,110",False
Canyonlands,"{'acres': '337,597.83', 'square_km': '1,366.2'}","{'latitude': 38.2, 'longitude': -109.93}","September 12, 1964",This landscape was eroded into a maze of canyo...,park_canyonlands,"776,218",False


### Extract google places Reviews


In [28]:
# use Park DataFrame for names
target_search = "yosemite national park"

params = {    
    "input": target_search,
    "inputtype": "textquery",
    "key": gkey
}

base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"


response = requests.get(base_url, params=params)

In [29]:
google_parks_data = response.json()
pprint(google_parks_data)

{'candidates': [{'place_id': 'ChIJxeyK9Z3wloAR_gOA7SycJC0'}], 'status': 'OK'}


In [31]:
params2 = {    
    "place_id":'ChIJxeyK9Z3wloAR_gOA7SycJC0',
    "key": gkey
}


base_detail_url= "https://maps.googleapis.com/maps/api/place/details/json"

response2 = requests.get(base_detail_url, params=params2)



In [32]:
yosemite_data=response2.json()
pprint(yosemite_data)

{'html_attributions': [],
 'result': {'address_components': [{'long_name': 'California',
                                    'short_name': 'CA',
                                    'types': ['administrative_area_level_1',
                                              'political']},
                                   {'long_name': 'United States',
                                    'short_name': 'US',
                                    'types': ['country', 'political']}],
            'adr_address': '<span class="region">California</span>, <span '
                           'class="country-name">USA</span>',
            'business_status': 'OPERATIONAL',
            'formatted_address': 'California, USA',
            'formatted_phone_number': '(209) 372-0200',
            'geometry': {'location': {'lat': 37.8651011, 'lng': -119.5383294},
                         'viewport': {'northeast': {'lat': 38.0831755,
                                                    'lng': -119.2172112},
      

In [36]:
print(yosemite_data["result"]["name"])
print(yosemite_data["result"]["rating"])
print(yosemite_data["result"]["user_ratings_total"])

Yosemite National Park
4.8
34183


### Transform Yelp! Data

### Create Database Connection

### Load DataFrames into Database